In [ ]:
## todo fix classes and labels code in detection.py
##tflite tends to give garbage classes to null detections

# INFERENCE - TEST MODE

In [1]:
import os
from streamutils import *
from matplotlib import pyplot as plt

def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)

def anonymize_face_pixelate(image, blocks=3):
	# divide the input image into NxN blocks
	(h, w) = image.shape[:2]
	xSteps = np.linspace(0, w, blocks + 1, dtype="int")
	ySteps = np.linspace(0, h, blocks + 1, dtype="int")
	# loop over the blocks in both the x and y direction
	for i in range(1, len(ySteps)):
		for j in range(1, len(xSteps)):
			# compute the starting and ending (x, y)-coordinates
			# for the current block
			startX = xSteps[j - 1]
			startY = ySteps[i - 1]
			endX = xSteps[j]
			endY = ySteps[i]
			# extract the ROI using NumPy array slicing, compute the
			# mean of the ROI, and then draw a rectangle with the
			# mean RGB values over the ROI in the original image
			roi = image[startY:endY, startX:endX]
			(B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
			cv2.rectangle(image, (startX, startY), (endX, endY),
				(B, G, R), -1)
	# return the pixelated blurred image
	return image


def streaming():
    
    config=readConfig()
    DB().createDB()
    
    model_kwargs=dict(config['model'])
    
    record=config['video']['record']
    draw=config['video']['draw']
    display=config['video']['display']
    test=config['video']['test']
    color=config['image']['color']
    
    
    #test_video=config['video']['test_video'] if test is not None else None
    test_video=config['video']['test_video'] if test else None
    
    schedule.every().second.do(DB().limitDB,config['db']['max_days']) 
    
    track,detection=True,True
    if not draw:
        track,detection=False,False
    
    #initialize model detection
    if detection:
        det=Detection()
        det.load_model(**model_kwargs)
        resolution=det.resolution
        
        if track:
            motrk=MotionTracker(config['motion']['frequency'])
            trk=Tracking(resolution)
     
    if record:
        vid,output_path=recorder(test_video,model_kwargs['model_path'],resolution)
        
    
    for frame,fps in stream_video(test_video,resolution):
        config=readConfig()
        det_kwargs=dict(config['detection'])
        counter_kwargs=dict(config['counter'])
        track_kwargs=dict(config['tracking'])
        
        
        frame,image=det.transform_image(frame,resolution,color)
        
        
        if detection:
            if not motrk.check_count():


                    results=det.model_inference(image)

                    results=det.filter_boxes(results,**det_kwargs)

                    results=det.scale_boxes(results,resolution)
                    
                    if config['motion']['frequency']>0:
                        motrk.start_trackers(image,results[0])

            else:

                results=motrk.update_trackers(image)
            
        
        if detection and track:
            
            trk.track_objects(results[0],counter_kwargs['line_points'],**track_kwargs)
           
            
            (on_update,output_data)=trk.update_counter(config['location']['capacity'],counter_kwargs['entrance'],
                                                       counter_kwargs['minutes_inactive'],  
                               counter_kwargs['percent_cap'],counter_kwargs['min_wait_time'],counter_kwargs['max_wait_time'],
                               counter_kwargs['reset'])
            
            if on_update:
                
                DB().writeDbData(output_data)

            
        #if fps is not None:
        #    fps.stop()
        #    curr_fps=round(fps.fps(),2)
        apply_filter=True
        if apply_filter:
            frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            frame=1-frame
            frame=adjust_gamma(frame,gamma=0.6)
            #frame=anonymize_face_pixelate(frame,110)
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
            
        

        
        if record or display:
            frame=det.draw_boxes(frame,results,det.labels)    
            
            if track:
                frame=trk.draw_tracking(frame)

            if record:
                vid.write(frame)
                
            if display:
                cv2.imshow('imgs',frame)
            
        if cv2.waitKey(1) == ord('q'):break 
            
        schedule.run_pending() 
                
    if record:
        vid.release()
        
        output_split=os.path.splitext(output_path)
        new_output_path='{}_{:.1f}{}'.format(output_split[0],fps.fps(),output_split[-1])
        os.rename(output_path,new_output_path)
     
        
    cv2.destroyAllWindows()


streaming()
            

<class 'numpy.float32'>


KeyboardInterrupt: 

In [ ]:
import numpy as np
import cv2
def anonymize_face_pixelate(image, blocks=3):
	# divide the input image into NxN blocks
	(h, w) = image.shape[:2]
	xSteps = np.linspace(0, w, blocks + 1, dtype="int")
	ySteps = np.linspace(0, h, blocks + 1, dtype="int")
	# loop over the blocks in both the x and y direction
	for i in range(1, len(ySteps)):
		for j in range(1, len(xSteps)):
			# compute the starting and ending (x, y)-coordinates
			# for the current block
			startX = xSteps[j - 1]
			startY = ySteps[i - 1]
			endX = xSteps[j]
			endY = ySteps[i]
			# extract the ROI using NumPy array slicing, compute the
			# mean of the ROI, and then draw a rectangle with the
			# mean RGB values over the ROI in the original image
			roi = image[startY:endY, startX:endX]
			(B, G, R) = [int(x) for x in cv2.mean(roi)[:3]]
			cv2.rectangle(image, (startX, startY), (endX, endY),
				(B, G, R), -1)
	# return the pixelated blurred image
	return image

def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)


cap = cv2.VideoCapture('videos/video-4.mp4')

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorMOG2()

while(1):
    ret, frame = cap.read()

    
    
    edges = cv2.Canny(frame,50,200)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #sobelx = cv2.Laplacian(gray,cv2.CV_64F)
    #sobelx =cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=1)
    
    
    anonymous=anonymize_face_pixelate(gray.copy(),50)
    
    
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    
    #fgmask=gray+edges
    
    
    
    fgmask=1-gray
    fgmask=adjust_gamma(fgmask,gamma=0.5)
    fgmask=anonymize_face_pixelate(fgmask,110)
    #fgmask+=anonymous
    
    cv2.imshow('frame',fgmask)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
def adjust_gamma(image, gamma=1.0):
	# build a lookup table mapping the pixel values [0, 255] to
	# their adjusted gamma values
	invGamma = 1.0 / gamma
	table = np.array([((i / 255.0) ** invGamma) * 255
		for i in np.arange(0, 256)]).astype("uint8")
	# apply gamma correction using the lookup table
	return cv2.LUT(image, table)


import cv2
import numpy as np
cap = cv2.VideoCapture("videos/video-4.mp4")

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

while(1):
    ret, frame2 = cap.read()
    
    
    #fgmask=anonymize_face_pixelate(fgmask,110)
    
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    next=1-next
    next=adjust_gamma(next,gamma=0.5)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    
    rgb=cv2.cvtColor(next,cv2.COLOR_GRAY2BGR)#+rgb

    cv2.imshow('frame2',rgb)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',rgb)
    prvs = next

cap.release()
cv2.destroyAllWindows()

In [4]:
## For converting mobnet3 to a valid inference graph
2**16

65536

In [6]:
resolution=(640,480)
angle=0

line_points=[[0, int(2*(resolution[1]/5))-angle],[resolution[0], int(2*(resolution[1]/5))+angle]]
line_points[0][0]=line_points[0][0]/resolution[0]
line_points[0][1]=line_points[0][1]/resolution[1]
line_points[1][0]=line_points[1][0]/resolution[0]
line_points[1][1]=line_points[1][1]/resolution[1]
line_points

[[0.0, 0.4], [1.0, 0.4]]

In [ ]:
68.667==18 in




In [1]:
68.667/18

3.8148333333333335

In [2]:
3.81*18

68.58

In [3]:
18/68.667

0.2621346498317969

In [12]:
ls=[0,1]
for l in zip(ls,ls):
    print(l)

(0, 0)
(1, 1)


[(0, 1)]